# KoSBERT 사용법
1. [현재 리포지토리](https://github.com/KAIST-Lawgical/Sentence-Embedding-Is-All-You-Need)를 `git clone`을 통해 받는다.
2. Lawgical KAIST의 구글 드라이브 내의 `Checkpoint` 폴더를 다운로드하여 `Sentence-Embedding-Is-All-You-Need` 바로 아래에 둔다.
3. 디렉토리 `Sentence-Embedding-Is-All-You-Need`에서 명령어 `docker build -t sbert:1.0.0 .`를 실행하여 도커 이미지를 빌드한다.
4. 명령어 `docker run -itd --ipc host --gpus all --name sbert sbert:1.0.0`를 도커 컨테이너를 실행한다.
5. `Visual Studio Code`의 `remote explorer`등을 통해 도커 컨테이너에 접속한다.
6. 도커 컨테이너 내부의 워크 디렉토리에서 이 파일, 즉 `test.ipynb`를 찾아 아래 코드를 활용하여 준비서면-판결문 문장 간 유사도 측정 작업을 실행한다.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model_path = './Sentence-Embedding-Is-All-You-Need/Checkpoint/KoSBERT/kosbert-klue-bert-base/'

embedder = SentenceTransformer(model_path)

# Corpus: 판결문 문장 리스트
corpus = [
    '1. 피고는 원고에게 12,000,000원 및 이에 대하여 2022. 3. 22.부터 2022. 7. 21.까지는 연 4%, 그 다음날부터 다 갚는 날까지는 연 12%의 각 비율로 계산한 돈을 지급하라.',
    '2. 원고의 나머지 청구를 기각한다.',
    '3. 소송비용은 각자 부담한다.',
    '4. 제1항은 가집행 할 수 있다.',
    '원고의 이 사건 청구는 위 인정범위 내에서 이유 있으므로 이를 인용하고, 나머지 청구는 이유 없으므로 이를 기각하기로 하여 주문과 같이 판결한다.'
]

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query: 준비서면 문장 리스트
queries = [
    '위 당사자 사이의 임대차보증금반환 청구사건에 관하여 원고들 소송대리인은 다음과 같이 변론을 준비합니다.',
    '이 사건 청구가 부적법하다거나, 이 사건 공제합의, 이 사건 채권압류·전부명령을 이유로 원고들의 각 청구를 거부하는 피고의 주장은 모두 부당합니다.',
    '피고는 원고들에게 이 사건 임대차보증금 2억 1,000만 원에서 2개월분 연체차임 3,000만 원을 제외한 나머지 1억 8,000만 원 중 원고들의 각 지분인 6,000만 원 및 이에 대하여 이 사건 소장 부본 송달일 다음 날부터 다 갚는 날까지 소송촉진 등에 관한 특례법에서 정한 연 12%의 비율로 계산한 지연손해금을 각각 지급할 의무가 있습니다.'
]

# Query의 각 문장에 대해 Corpus의 문장 중 embedding의 cosine similarity 기준으로 가장 유사한 top_k개의 문장과 유사도를 반환
top_k = 1
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {} most similar sentences in corpus:".format(top_k))

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
